In [ ]:
!nvidia-smi

In [ ]:
pip install transformers

In [ ]:
import pandas as pd
import re
import spacy

In [ ]:
df1 = pd.read_csv("Dataset1.csv")
df2 = pd.read_csv("Dataset2.csv")
df2.drop('Timestamp',axis=1,inplace=True)
df2.columns = ['Tell me about yourself',
       'Tell me about a time you demonstrated leadership',
       'Tell me about a time when your working on a team and faced with a challenge how did you solve that problem?',
       'tell me about one of your weaknesses and how you plan to overcome it.',
       'Now why do you think we should hire you?', 'Label']
df = df1.append(df2,ignore_index=True)
#df = pd.read_csv("Dataset3.csv")
df.shape

(162, 6)

In [ ]:
df.head()

In [ ]:
from spacy.lang.en.stop_words import STOP_WORDS
def clean(mytext):
  tokens = []
  nlp = spacy.load('en',parser=False,entity=False)

  #spacy customize stopwords  
  # New stop words list 
  customize_stop_words = [
      '.',
      '-',
      '.....',
      '....'
      ]
  # Mark them as stop words
  for w in customize_stop_words:
      nlp.vocab[w].is_stop = True

  my_doc = nlp(mytext)
  for token in my_doc:
    if not token.is_stop:
      tokens.append(token.lemma_)
  
  return tokens

In [ ]:
mylist = []
for i in range(len(df)):
  text = ''
  for j in range(len(df.iloc[i])-1):
    text+=re.sub(r'\|Interviewe(r|e): | \[.*\] | \(.*\) | \{.*\} | \.*',' ',df.iloc[i][j],flags=re.IGNORECASE)
  mylist.append((df.iloc[i][5]," ".join(clean(text))))


In [ ]:
mydf = pd.DataFrame(mylist,columns=['Label','Reponses'])

In [ ]:
mydf.shape

(162, 2)

In [ ]:
#try a BERT model
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import torch
import transformers as ppb
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import tensorflow as tf
tf.test.gpu_device_name()

''

In [ ]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

In [ ]:
#divide into train test spilt
def load_data(dataframe,split_ratio):
  #shuffle the data
  data = dataframe.sample(frac=1,random_state=7)

  texts = data.Reponses.values
  labels = [{'cats': {'2': label == 2,
                      '3': label == 3,
                      '4': label == 4,
                      '5': label == 5
                      }
            } for label in data['Label']]

  split = int(len(data) * split_ratio)
    
  return texts[:split], labels[:split], texts[split:], labels[split:]

train_data,train_labels,val_data,val_labels = load_data(mydf,0.8)

print(train_data.shape,val_data.shape)

(129,) (33,)


In [ ]:
temp

In [ ]:
temp = mydf['Label']
temp = list(temp)
classes = 4
encoded = np.eye(classes)[temp]
encoded

IndexError: ignored

In [ ]:
from numpy import array
from sklearn.preprocessing import OneHotEncoder
labels = mydf['Label']
data = array(labels)

encoded = [[]]


162


In [ ]:
train_zip = list(zip(train_data, train_labels))
val_zip = list(zip(val_data, val_labels))

In [ ]:
#spacy supervised model
nlp = spacy.blank("en")
# Create the TextCategorizer with exclusive classes and "bow" architecture
textcat = nlp.create_pipe(
              "textcat",
              config={
                "exclusive_classes": True,
                "architecture": "bow"})

# Add the TextCategorizer to the empty model
nlp.add_pipe(textcat)

In [ ]:
# Add labels to text classifier
textcat.add_label("2")
textcat.add_label("3")
textcat.add_label("4")
textcat.add_label("5")

1

In [ ]:
import random
from spacy.util import minibatch

random.seed(1)
spacy.util.fix_random_seed(1)
optimizer = nlp.begin_training()

losses = {}
for epoch in range(20):
    random.shuffle(train_zip)
    # Create the batch generator with batch size = 8
    batches = minibatch(train_zip, size=8)
    # Iterate through minibatches
    for batch in batches:
        # Each batch is a list of (text, label) but we need to
        # send separate lists for texts and labels to update().
        # This is a quick way to split a list of tuples into lists
        texts, labels = zip(*batch)
        nlp.update(texts, labels, sgd=optimizer, losses=losses)
    print(losses)

{'textcat': 1.6318165543489158}
{'textcat': 2.042804362718016}
{'textcat': 2.0749434651133924}
{'textcat': 2.1790675607826415}
{'textcat': 2.2459526215770893}
{'textcat': 2.2519719772972167}
{'textcat': 2.253616067171606}
{'textcat': 2.2579116714653082}
{'textcat': 2.2580348983043095}
{'textcat': 2.264004411544782}
{'textcat': 2.264067159791942}
{'textcat': 2.264437456205421}
{'textcat': 2.2644620694440505}
{'textcat': 2.264507842774549}
{'textcat': 2.264523770080075}
{'textcat': 2.264632078150118}
{'textcat': 2.264642749306997}
{'textcat': 2.264651459505952}
{'textcat': 2.2646682447263053}
{'textcat': 2.26467409677164}


In [ ]:
#try saving the model
nlp.to_disk('/content/myModel')

In [ ]:
#load the model
myModel = spacy.load('/content/myModel')

In [ ]:
def predict(nlp,texts):
  docs = [nlp.tokenizer(text) for text in texts]
  textcat = nlp.get_pipe('textcat')
  score,tensor = textcat.predict(docs)
  predicted_class = score.argmax(axis=1)
  return predicted_class

In [ ]:
def evaluate(model,texts,labels):
  predicted_class = predict(model,texts)
  true_class = []
  for i in range(len(labels)):
    true_class.append(list(labels[i]['cats'].values()).index(True))

  #print(true_class[:10],predicted_class[:10])

  correct_predictions = predicted_class == true_class

  #print(correct_predictions[:10])

  return correct_predictions.mean()

In [ ]:
accuracy = evaluate(nlp, val_data, val_labels)
print(f"Accuracy: {accuracy:.4f}")

In [ ]:
#bow = 0.5848
#ensemble = 0.3636
#CNN = 0.4545

val_labels[:10]

In [ ]:
texts = val_data[:10]
predictions = predict(myModel1, texts)

for p, t in zip(predictions, texts):
    print(f"{textcat.labels[p]}: {t} \n")

In [ ]:
#shows the probabilities for each class
mydoc = nlp(to_predict)
mydoc.cats

{'1': 0.0021423092111945152,
 '2': 0.003565057646483183,
 '3': 0.9886618852615356,
 '4': 0.0037832744419574738,
 '5': 0.001847579376772046}

In [ ]:
first = ["My name is Vedant and I'm 21 years old. I am in my last year of engineering. I love to uhmm play sports, read books, watch series and movies , and listen to songs in my spare time."]
second = ["uhm...This happened about 3 years back when I was in FE. We were playing quarter finals of a tournament and we're comfortably leading 2-0. All of a sudden our team lost concentration and we conceded 2 goals in 5'. Everyone seemed demotivated. This is when I tried to motivate them and in trying to do so scored a goal which for their spirits back up. We ended up winning the game 4-2."]
third = ["It was our mini project submission and we were getting an error. After calling up around 5-6 friends my team and I couldn't solve the error. So we were thinking of restarting the whole project and I just told them to calm down and look at the code once again. We found the root of the error and thankfully it got solved."]
fourth = ["One weakness I've is I get short tempered when we're working as a team and I don't see 100% efforts from my  team to win a match/ complete a task. I'm of the opinion that efforts matter more than results. But after thinking alot I feel that sometimes results are as important as efforts. So I'm trying to overcome this thing."]
fifth = ["I possess all the skills and experience that you're looking for. I'm pretty confident that I am the best candidate for this job role. It's not just my background in the past projects, but also my people skills, which will be applicable in this position. With my experience in extra curricular I might even be able to give a different angle to some situations"]

temp = first + second + third + fourth + fifth
testText = " ".join(temp)
testText

"My name is Vedant and I'm 21 years old. I am in my last year of engineering. I love to uhmm play sports, read books, watch series and movies , and listen to songs in my spare time. uhm...This happened about 3 years back when I was in FE. We were playing quarter finals of a tournament and we're comfortably leading 2-0. All of a sudden our team lost concentration and we conceded 2 goals in 5'. Everyone seemed demotivated. This is when I tried to motivate them and in trying to do so scored a goal which for their spirits back up. We ended up winning the game 4-2. It was our mini project submission and we were getting an error. After calling up around 5-6 friends my team and I couldn't solve the error. So we were thinking of restarting the whole project and I just told them to calm down and look at the code once again. We found the root of the error and thankfully it got solved. One weakness I've is I get short tempered when we're working as a team and I don't see 100% efforts from my  tea

In [ ]:
to_predict = " ".join(clean(testText))

In [ ]:
import cv2
import os

In [ ]:
classifier_path_face = '/usr/local/lib/python3.7/dist-packages/cv2/data/haarcascade_frontalface_alt.xml'
classifier_path_eye = '/usr/local/lib/python3.7/dist-packages/cv2/data/haarcascade_eye.xml'
image_path = '/content/apala2.jpeg'

In [ ]:
from google.colab.patches import cv2_imshow

def face_eye_detection(image_path,face_classifier_path,eye_classifier_path):
  original_image = cv2.imread(image_path)
  gray_image = cv2.cvtColor(original_image,cv2.COLOR_BGR2GRAY)

  face_classifier = cv2.CascadeClassifier(face_classifier_path)
  eye_classifier = cv2.CascadeClassifier(eye_classifier_path)

  faces = face_classifier.detectMultiScale(gray_image,1.3,5)

  for (column,row,width,height) in faces:
    cv2.rectangle(original_image,(column,row),(column+width,row+height),(0,255,0),2)

    #roi_color = original_image[row:row+height,column:column+width]
    #roi_gray = gray_image[row:row+height,column:column+width]

    #face_gray_image = cv2.cvtColor(face_image,cv2.COLOR_BGR2GRAY)
    eyes = eye_classifier.detectMultiScale(gray_image)
    for (ecolumn,erow,ewidth,eheight) in eyes:
      cv2.rectangle(original_image,(ecolumn,erow),(ecolumn+ewidth,erow+eheight),(255,0,0),2)
  
  fname, ext = os.path.splitext(image_path)
  cv2.imwrite(fname+"_cropped"+ext,original_image)
  #cv2_imshow(original_image)
  return


In [ ]:
face_eye_detection(image_path,classifier_path_eye,classifier_path_face)

In [ ]:
import os
fname,ext = os.path.splitext(image_path)
print(fname,ext)

/content/jokers .png


In [ ]:
def detectface(image,classifier):
  #read the image
  original_image = cv2.imread(image)

  #convert it to grayscale
  gray_scale = cv2.cvtColor(original_image,cv2.COLOR_BGR2GRAY)

  #load the classifier
  face_cascade = cv2.CascadeClassifier(classifier)
  detected_faces = face_cascade.detectMultiScale(gray_scale)
  #counter = 0

  if detected_faces is ():
    print('No face detected')
  else:
    for (column,row,width,height) in detected_faces:
      image = cv2.rectangle(original_image,(column,row),(column+width,row+height),(0,255,0),2)
      fname, ext = os.path.splitext(image_path)
      cv2.imwrite(fname+"_detected"+str(counter)+ext,image)
      #counter+=1
  return


In [ ]:
detectface(image_path,classifier_path_face)

No face detected


In [ ]:
classifier_path_eye = '/usr/local/lib/python3.7/dist-packages/cv2/data/haarcascade_eye.xml'
def detecteyes(image,classifier):
  original_image = cv2.imread(image)
  gray_scale = cv2.cvtColor(original_image,cv2.COLOR_BGR2GRAY)

  eye_cascade = cv2.CascadeClassifier(classifier)
  detected_eyes = eye_cascade.detectMultiScale(gray_scale)

  
  for (column,row,width,height) in detected_eyes:
    print(column,row,width,height)
    image = cv2.rectangle(original_image,(column,row),(column+width,row+height),(0,255,0),2)
    fname, ext = os.path.splitext(image_path)
    cv2.imwrite(fname+"_detected"+ext,image)

  return
  

In [ ]:
detecteyes(image_path,classifier_path_eye)

209 209 25 25
255 209 27 27
351 242 29 29
474 239 32 32
385 244 31 31
519 241 34 34
